In [29]:
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# ================= 配置区域 =================
ROBOT_CSV = 'raw_data/robot_poses.csv'     # Base -> Gripper (x,y,z,qx,qy,qz,qw)
VISION_CSV = 'raw_data/vision_poses.csv'   # Target -> Camera (tx,ty,tz, rx,ry,rz)

CALIB_METHOD = cv2.CALIB_HAND_EYE_TSAI
# ===========================================


def run_calibration():
    print("--- 开始手眼标定计算 (Eye-to-Hand) ---")

    # 1. 读取数据
    df_robot = pd.read_csv(ROBOT_CSV)
    df_vision = pd.read_csv(VISION_CSV)

    print(f"读取到机械臂数据: {len(df_robot)} 行")
    print(f"读取到视觉数据: {len(df_vision)} 行")

    # 2. OpenCV 所需的数据容器
    R_gripper2base = []   # Gripper -> Base  ❗
    t_gripper2base = []
    R_target2cam = []     # Target -> Camera
    t_target2cam = []

    valid_count = 0

    # 3. 数据配对
    for _, row_vision in df_vision.iterrows():
        img_idx = int(row_vision['img_index'])

        if img_idx >= len(df_robot):
            continue

        # ===== A. 机械臂数据 =====
        # CSV 中存的是 Base -> Gripper
        row_robot = df_robot.iloc[img_idx]

        t_bg = np.array([
            row_robot['x'],
            row_robot['y'],
            row_robot['z']
        ]).reshape(3, 1)

        q = [
            row_robot['qx'],
            row_robot['qy'],
            row_robot['qz'],
            row_robot['qw']
        ]

        # Base -> Gripper
        R_bg = R.from_quat(q).as_matrix()

        # ❗ 关键修正：求逆，得到 Gripper -> Base
        R_gb = R_bg.T
        t_gb = -R_bg.T @ t_bg

        # ===== B. 视觉数据 =====
        # 你明确说明：棋盘格在相机坐标系下（Target -> Camera）
        t_tc = np.array([
            row_vision['cam_tx'],
            row_vision['cam_ty'],
            row_vision['cam_tz']
        ]).reshape(3, 1)

        rvec = np.array([
            row_vision['cam_rx'],
            row_vision['cam_ry'],
            row_vision['cam_rz']
        ])

        R_tc, _ = cv2.Rodrigues(rvec)

        # ===== C. 存入 =====
        R_gripper2base.append(R_gb)
        t_gripper2base.append(t_gb)
        R_target2cam.append(R_tc)
        t_target2cam.append(t_tc)

        valid_count += 1

    print(f"有效配对数据: {valid_count} 组")
    if valid_count < 3:
        print("❌ 数据不足，无法标定")
        return

    # 4. 执行手眼标定
    R_base_cam, t_base_cam = cv2.calibrateHandEye(
        R_gripper2base=R_gripper2base,
        t_gripper2base=t_gripper2base,
        R_target2cam=R_target2cam,
        t_target2cam=t_target2cam,
        method=CALIB_METHOD
    )

    # 5. 组装齐次矩阵
    T_base_cam = np.eye(4)
    T_base_cam[:3, :3] = R_base_cam
    T_base_cam[:3, 3] = t_base_cam.flatten()

    T_cam_base = np.linalg.inv(T_base_cam)

    # 6. 输出结果
    np.set_printoptions(precision=6, suppress=True)

    print("\n✅ 标定成功")
    print("\nT_base_cam (Base -> Camera):")
    print(T_base_cam)

    print("\nT_cam_base (Camera -> Base):")
    print(T_cam_base)

    # 平移与欧拉角
    print("\n结果解读:")
    print("平移 (m):", t_base_cam.flatten())

    euler = R.from_matrix(R_base_cam).as_euler('xyz', degrees=True)
    print("旋转 (deg, xyz):", euler)

    # 7. 保存
    np.savetxt("result_hand_eye.txt", T_base_cam, fmt="%.8f")
    print("\n结果已保存至 result_hand_eye.txt")


if __name__ == "__main__":
    run_calibration()


--- 开始手眼标定计算 (Eye-to-Hand) ---
读取到机械臂数据: 25 行
读取到视觉数据: 24 行
有效配对数据: 24 组

✅ 标定成功

T_base_cam (Base -> Camera):
[[ 0.997843 -0.01868   0.062937 -0.489227]
 [-0.050873 -0.825963  0.561425 -0.612413]
 [ 0.041496 -0.563415 -0.825131  0.505419]
 [ 0.        0.        0.        1.      ]]

T_cam_base (Camera -> Base):
[[ 0.997843 -0.050873  0.041496  0.436044]
 [-0.01868  -0.825963 -0.563415 -0.230208]
 [ 0.062937  0.561425 -0.825131  0.791651]
 [ 0.        0.        0.        1.      ]]

结果解读:
平移 (m): [-0.489227 -0.612413  0.505419]
旋转 (deg, xyz): [-145.674      -2.378246   -2.918561]

结果已保存至 result_hand_eye.txt
